Scraping posts from Reddit threads


1. r/WallStreetBets
2. r/Stocks
3. r/Investing

Reference:
- (ticker code)
https://medium.com/@tom.santinelli/scraping-reddits-wall-street-bets-for-popular-stock-tickers-38ed5202affc
- (stock tickers)
https://www.nasdaq.com/market-activity/stocks/screener
- (Flair NLP)
https://github.com/flairNLP/flair
- (NLP code)
https://towardsdatascience.com/sentiment-analysis-for-stock-price-prediction-in-python-bed40c65d178
- https://github.com/dmarx/psaw


Dependencies and initialization
=========

In [ ]:
!pip install praw
!pip install psaw
!pip install flair
import flair
import praw
import pandas as pd
import re  
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from psaw import PushshiftAPI


     |████████████████████████████████| 163kB 36.1MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 286kB 29.6MB/s 
     |████████████████████████████████| 19.7MB 1.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2MB 44.8MB/s 
     |████████████████████████████████| 798kB 49.2MB/s 
     |████████████████████████████████| 2.3MB 37.2MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 983kB 38.1MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 901kB 43.4MB/s 
     |████████████████████████████████| 3.3MB 42.8MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=7a007183d1cee71ed7214b73bc4a17cb0a32f8a2ea1e786dd7120a4f5273d8fc
  

Declare a reddit instance (used a dummy account)

In [ ]:
reddit = praw.Reddit(client_id='6pKo4YmL2zkS0Q', \
                     client_secret='kCZTMPNF7Do43YNPbWlWihtbQFG0cg', \
                     user_agent='scraper', \
                     )

Declare subredit instances: wallstreetbets, stocks, investing

In [ ]:
wsb_subr = reddit.subreddit('wallstreetbets')
stocks_subr = reddit.subreddit('stocks')
invest_subr = reddit.subreddit('investing')

Generating the most commonly mentioned stock tickers
========

1. Grabbing the title and content of each post and appending to a dataframe

In [ ]:
df = []

for post in wsb_subr.hot(limit = 500):
    content = {
        "title": post.title,
        "text": post.selftext
    }
    df.append(content)

for post in stocks_subr.hot(limit = 500):
    content = {
        "title": post.title,
        "text": post.selftext
    }
    df.append(content)

for post in invest_subr.hot(limit = 500):
    content = {
        "title": post.title,
        "text": post.selftext
    }
    df.append(content)

df = pd.DataFrame(df)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

2. Regular expression for filtering 

In [ ]:
regex = re.compile('[^a-zA-Z ]')

3. Analyze word frequency

In [ ]:
word_dict = {}

for (index, row) in df.iterrows():
    
    # title
    title = row['title']
    title = regex.sub('',title)
    title_words = title.split(' ')

    # content 
    content = row['text']
    content = regex.sub('',content)
    content_words = content.split(' ')

    # combine
    words = title_words + content_words
    for x in words:
        if x in ['A', 'B', 'GO', 'ARE', 'ON']:
            pass
        elif x in word_dict:
            word_dict[x] += 1
        else:
            word_dict[x] = 1


word_df = pd.DataFrame.from_dict(list(word_dict.items())).rename(columns = {0:"Term",1:"Frequency"})

4. Read the csv of stock tickes into a dataframe

In [ ]:
ticker_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CMSC472_2021_Final_Project/tickers.csv').rename(columns = {'Symbol':'Term', 'Name':'Company_Name'})

5. Inner join of the two datasets, and then sort by the frequency they are mentioned from. 

In [ ]:
stonks_df = pd.merge(ticker_df, word_df, on="Term")
stonks_df = stonks_df.sort_values(by = "Frequency",ascending=False)

In [ ]:
idx = 0;
chosen_comp = [];
for t in stonks_df['Term']:
    chosen_comp.append(t)

chosen_comp = chosen_comp[0:3]
chosen_comp



['M', 'DD', 'K']

Now stonks_df is a list of stock tickers and their frequency in 500 hot wall street bets, stocks, and investing posts. 

Retrieving the posts that mentioned the top 10 stock tickers
==========


1. Adding a column "Content" into the stonks_df dataframe 

In [ ]:
if stonks_df.__contains__("Content") == False:
    stonks_df.insert(len(stonks_df.columns), "Content", " ")

2. For the top 10 mentioned stocks, retrieve all the comments and titles that mentioned it and store in the corresponding "Content" section

In [ ]:
regex = re.compile('[^a-zA-Z. ]')
whitespace = re.compile(r"\s+")
web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")

In [ ]:
for i in range(10):
    t = stonks_df['Term'][i]


    for (index, row) in df.iterrows():
        
        # title
        title = row['title']
        title = regex.sub(' ',title)
        title = whitespace.sub(' ', title)
        title_sentence = title.split('.')
        title_words = title.split(' ')

        # content 
        content = row['text']
        content = web_address.sub(' ', content)
        content = regex.sub(' ',content)
        content = whitespace.sub(' ', content)
        content_sentence = content.split('.')
        content_words = content.split(' ')

        # combine
        sentence = title_sentence + content_sentence
        
        words = title_words + content_words
        for x in words:
            if x == t and sentence != " " and sentence != "":
                stonks_df.at[i,'Content'] = sentence
                break

Now, we are ready to run NLP with each of the contents associated with top mentioned stocks

NLP with the comments
==========

1. Install Flair for NLP

In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 6.9MB/s 
     |████████████████████████████████| 983kB 8.5MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 1.2MB 44.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.1MB 47.6MB/s 
     |████████████████████████████████| 71kB 4.1MB/s 
     |████████████████████████████████| 798kB 44.4MB/s 
     |████████████████████████████████| 3.3MB 38.8MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=154e5e441c2af83233216a385c2056f18305d486b542105f91d0c78ce4e42624
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for segtok: filename=

2. Initialize flair

In [ ]:
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-05-10 00:14:29,254 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpobs9s1yf


100%|██████████| 265512723/265512723 [00:16<00:00, 16219181.40B/s]

2021-05-10 00:14:46,116 copying /tmp/tmpobs9s1yf to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-05-10 00:14:46,937 removing temp file /tmp/tmpobs9s1yf
2021-05-10 00:14:47,711 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


3. Add another column "Sentiment" into the dataframe 

In [ ]:
if stonks_df.__contains__("Sentiment") == False:
    stonks_df.insert(len(stonks_df.columns), "Sentiment", 0.00)

4. Generating sentiment score for the top stocks and store in the data fram

In [ ]:
for i in range(10):
    neg = 0
    pos = 0
    slen = 0
    for s in stonks_df["Content"][i]:
        s = s.strip()
        if s == "":
            slen -= 1
            continue

        s = flair.data.Sentence(s)
        sentiment_model.predict(s)
        
        score = s.labels[0].score
        value = s.labels[0].value
        if value == 'POSITIVE':
            pos += score
        else:  
            neg += score
        slen +=1
    total_score = (pos - neg) / slen
    stonks_df.at[i,'Sentiment'] = total_score



Combine the above into methods
==========

In [ ]:
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-05-14 19:44:00,471 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmposxqib2i


100%|██████████| 265512723/265512723 [00:09<00:00, 26784779.18B/s]

2021-05-14 19:44:10,703 copying /tmp/tmposxqib2i to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-05-14 19:44:11,823 removing temp file /tmp/tmposxqib2i
2021-05-14 19:44:13,002 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


** The method below outputs the top mentioned stock tickers from r/wallstreet, r/stocks, r/investing ** 

In [ ]:
# Input
# - ticker_df (a data frame that contains the ticker lists)
# - limit (the number of hot posts from each subreddit)
# output 
# - df (data frame that contains the number of times mentioned for each ticker)

def topMentionedReddit(ticker_df, limit):
    reddit = praw.Reddit(client_id='6pKo4YmL2zkS0Q', \
                     client_secret='kCZTMPNF7Do43YNPbWlWihtbQFG0cg', \
                     user_agent='scraper', \
                     )
    wsb_subr = reddit.subreddit('wallstreetbets')
    stocks_subr = reddit.subreddit('stocks')
    invest_subr = reddit.subreddit('investing')

    regex = re.compile('[^a-zA-Z. ]')
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")

    df = []
    for post in wsb_subr.hot(limit = limit):
        content = {
            "title": post.title,
            "text": post.selftext
        }
        df.append(content)

    for post in stocks_subr.hot(limit = limit):
        content = {
            "title": post.title,
            "text": post.selftext
        }
        df.append(content)

    for post in invest_subr.hot(limit = limit):
        content = {
            "title": post.title,
            "text": post.selftext
        }
        df.append(content)

    df = pd.DataFrame(df)

    regex = re.compile('[^a-zA-Z ]')
    word_dict = {}

    for (index, row) in df.iterrows():
        
        # title
        title = row['title']
        title = regex.sub('',title)
        title_words = title.split(' ')

        # content 
        content = row['text']
        content = regex.sub('',content)
        content_words = content.split(' ')

        # combine
        words = title_words + content_words
        for x in words:
            if x in ['A', 'B', 'GO', 'ARE', 'ON']:
                pass
            elif x in word_dict:
                word_dict[x] += 1
            else:
                word_dict[x] = 1


    word_df = pd.DataFrame.from_dict(list(word_dict.items())).rename(columns = {0:"Term",1:"Frequency"})
    #ticker_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CMSC472_2021_Final_Project/tickers.csv').rename(columns = {'Symbol':'Term', 'Name':'Company_Name'})

    stonks_df = pd.merge(ticker_df, word_df, on="Term")
    stonks_df = stonks_df.sort_values(by = "Frequency",ascending=False)

    return stonks_df

** The method below looks up a specific stock ticker within the given time frame, scrap the posts that mention the specific stock on the day and analyze the sentiment score for the day **

In [ ]:
# Input: 
# - ticker (string)
# - begin_date (datetime object)
# - end_date(datetime object)
# - verbose (Boolean) True for for printing results; false otherwise
# output:
# - cvs file that contains the date and the sentiment score from the given time frame

def redditDailyScore(ticker, begin_date, end_date, verbose):
    api = PushshiftAPI()
    day_count = (end_date - begin_date).days

    regex = re.compile('[^a-zA-Z. ]')
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")

    dateDF = pd.DataFrame();
    dateDF.insert(0,'Date',0)
    dateDF.insert(1, "Sentiment",0)
    dateIdx = 0;

    for single_date in (begin_date + dt.timedelta(n) for n in range(day_count)):
        
        try:
            after =int(single_date.timestamp())
            before = int((single_date +dt.timedelta(1)).timestamp())
            gen = api.search_submissions(before = before,
                                after = after,
                                q = ticker,        
                                filter = ['id','author','title','num_comments','selftext'],
                                subreddit='wallstreetbets',
                                limit=1000)
            
            gen2 = api.search_submissions(before = before,
                                after = after,
                                q = ticker,        
                                filter = ['id','author','title','num_comments','selftext'],
                                subreddit='stocks',
                                limit=1000)
            
            gen3 = api.search_submissions(before = before,
                                after = after,
                                q = ticker,        
                                filter = ['id','author','title','num_comments','selftext'],
                                subreddit='investing',
                                limit=1000)
            
            df = pd.DataFrame(gen)
            df2 = pd.DataFrame(gen2)
            df3 = pd.DataFrame(gen3)

            content = []
            

            for (index, row) in df.iterrows():
            
                # title
                title = row['title']
                title = regex.sub(' ',title)
                title = whitespace.sub(' ', title)
                title_sentence = title.split('.')

                # body
                body = row['selftext']
                body = web_address.sub(' ', body)
                body = regex.sub(' ',body)
                body = whitespace.sub(' ', body)
                body_sentence = body.split('.')

                # combine
                sentence = title_sentence + body_sentence
                content = np.append(sentence,content)
            
            for (index, row) in df2.iterrows():
            
                # title
                title = row['title']
                title = regex.sub(' ',title)
                title = whitespace.sub(' ', title)
                title_sentence = title.split('.')

                # body
                body = row['selftext']
                body = web_address.sub(' ', body)
                body = regex.sub(' ',body)
                body = whitespace.sub(' ', body)
                body_sentence = body.split('.')

                # combine
                sentence = title_sentence + body_sentence
                content = np.append(sentence,content)
            
            for (index, row) in df3.iterrows():
            
                # title
                title = row['title']
                title = regex.sub(' ',title)
                title = whitespace.sub(' ', title)
                title_sentence = title.split('.')

                # body
                body = row['selftext']
                body = web_address.sub(' ', body)
                body = regex.sub(' ',body)
                body = whitespace.sub(' ', body)
                body_sentence = body.split('.')

                # combine
                sentence = title_sentence + body_sentence
                content = np.append(sentence,content)
          

            neg = 0
            pos = 0
            slen = 0
            for s in content:
                if s == "":
                    continue

                if s == " ":
                    continue

                s = flair.data.Sentence(s)
                sentiment_model.predict(s)
                
                score = s.labels[0].score
                value = s.labels[0].value
                if value == 'POSITIVE':
                    pos += score
                else:  
                    neg += score
                slen +=1

            if slen == 0:
                total_score = 0
            else: 
                total_score = (pos - neg) / slen
            
          

            dateDF.at[dateIdx,'Date'] = single_date
            dateDF.at[dateIdx, 'Sentiment'] = total_score
            dateIdx += 1

            if verbose:
                s = str(single_date) + ' ' + str(total_score)
                print(s) 
        
        except:
            dateDF.at[dateIdx,'Date'] = single_date
            dateDF.at[dateIdx, 'Sentiment'] = 0
            dateIdx += 1
            if verbose:
                s = str(single_date) + ' ' + "Non-String"
                print(s)
            
            continue

    return dateDF



            
            
        


Use topMentionReddit to generate the top 10 tickers
==========



In [ ]:
ticker_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CMSC472_2021_Final_Project/tickers.csv').rename(columns = {'Symbol':'Term', 'Name':'Company_Name'})
stonks_df = topMentionedReddit(ticker_df,500)

In [ ]:
company = stonks_df['Term']
idx = 0;
chosen_comp = [];
for t in stonks_df['Term']:
    chosen_comp.append(t)

chosen_comp = chosen_comp[0:10]
chosen_comp


['M', 'DD', 'SP', 'K', 'UWMC', 'FOR', 'G', 'PLTR', 'INO', 'TSLA']

Use redditDailyScore to generate csv files that contains the sentiment score from 2019 to 2021
==========

In [ ]:
resultDF = redditDailyScore('M', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF.to_csv("M_2019.csv") #done

In [ ]:
resultDF2 = redditDailyScore('DD', dt.datetime(2019,1,1,0,0), dt.datetime(2019,10,1,0,0),True)
resultDF2.to_csv("DD_2019_1.csv") #done

In [ ]:
resultDF2_2 = redditDailyScore('DD', dt.datetime(2019,10,2,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF2_2.to_csv("DD_2019_2.csv") #done

In [ ]:
resultDF3 = redditDailyScore("K",dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF3.to_csv("K_2019.csv")#done

In [ ]:
resultDF_SP = redditDailyScore('SP', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_SP.to_csv("SP_2019.csv")#done

In [ ]:
resultDF_SP = redditDailyScore('UWMC', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_SP.to_csv("UWMC_2019.csv")#done

In [ ]:
resultDF_SP1 = redditDailyScore('FOR', dt.datetime(2019,1,1,0,0), dt.datetime(2019,5,6,0,0),True)
resultDF_SP1.to_csv("FOR_2019_1 .csv")#done

In [ ]:
resultDF_SP1.to_csv("FOR_2019_1 .csv")#done

In [ ]:
resultDF_FOR2 = redditDailyScore('FOR', dt.datetime(2019,5,8,0,0), dt.datetime(2019,7,2,0,0),True)
resultDF_FOR2.to_csv("FOR_2019_2 .csv")#done

In [ ]:
resultDF_FOR3 = redditDailyScore('FOR', dt.datetime(2019,7,2,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_FOR3.to_csv("FOR_2019_3 .csv")#done

In [ ]:
resultDF_G = redditDailyScore('G', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_G.to_csv("G_2019.csv")#done

In [ ]:
resultDF_PLTR = redditDailyScore('PLTR', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_PLTR.to_csv("PLTR_2019.csv")#done

In [ ]:
resultDF_TSLA = redditDailyScore('TSLA', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_TSLA.to_csv("TSLA_2019.csv")#done

In [ ]:
resultDF_SP500= redditDailyScore('SP500', dt.datetime(2019,1,1,0,0), dt.datetime(2020,1,1,0,0),True)
resultDF_SP500.to_csv("SP500_2019.csv")#done

In [ ]:
resultDF_M_1 = redditDailyScore('M', dt.datetime(2020,1,1,0,0), dt.datetime(2020,3,16,0,0),True)
resultDF_M_1.to_csv("M_2020_1.csv") #done #3/16 not array like object?

In [ ]:
resultDF_M_2 = redditDailyScore('M', dt.datetime(2020,3,17,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_M_2.to_csv("M_2020_2.csv") #done #3/16 not array like object?

In [ ]:
resultDF_DD = redditDailyScore('DD', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_DD.to_csv("DD_2020.csv") #done

In [ ]:
resultDF_SP = redditDailyScore('SP', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_SP.to_csv("SP_2020.csv") #done

In [ ]:
resultDF_K = redditDailyScore('K', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_K.to_csv("K_2020.csv") #done

In [ ]:
resultDF_FOR = redditDailyScore('FOR', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_FOR.to_csv("FOR_2020.csv") #done

In [ ]:
resultDF_G = redditDailyScore('G', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_G.to_csv("G_2020.csv") #done

In [ ]:
resultDF_TSLA = redditDailyScore('TSLA', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_TSLA.to_csv("TSLA_2020.csv") #done

In [ ]:
resultDF_SP500= redditDailyScore('SP500', dt.datetime(2020,1,1,0,0), dt.datetime(2021,1,1,0,0),True)
resultDF_SP500.to_csv("SP500_2019.csv")#done